In [1]:
#!pip install folium
#!pip install ipywidgets
import folium
from folium import plugins
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display
# path of bridge data for reading csv file (_roads2)
road_path     = 'D:\Jupyter_File\Advanced_Discrete_Cleaning\WBSIM_Lab2\infrastructure\_roads3.csv'

# path of bridge data for reading Excel file (BMMS_overview)
bridge_path     = 'D:\Jupyter_File\Advanced_Discrete_Cleaning\WBSIM_Lab2\infrastructure\BMMS_overview.xlsx'

# Set filepath (fix path relative to yours)
fp = 'D:/Jupyter_File/Advanced_Discrete_Cleaning/Assignment_3/bangladesh_floods_adm4'

# define variables
path  = 'D:\Jupyter_File\Advanced_Discrete_Cleaning\RMMS' # path to RMMS

In [2]:
def import_data(road_path,bridge_path,fp,path):    
    
    df_orig  = pd.read_csv(road_path,index_col=None, header=0)
    
    bdf_orig = pd.read_excel(bridge_path, index_col=None, header=0)
    
    flood_shhp_data = gpd.read_file(fp)
    
    flood_geoPath = flood_shhp_data.geometry.to_json
    
    rname0 = 'N1'
    textPath = path + '\\' + rname0 + '.widths.processed.txt'
    width_data0      = pd.read_table(textPath)

    # reading raw html
    htmlPath = path + '\\' + rname0 + ".traffic.htm"
    rawHtml0  = pd.read_html(htmlPath)[4]

    rawHtml0.loc[2,0] = 'LinkNo'
    rawHtml0.loc[2,1] = 'LinkName'
    rawHtml0.loc[2,4] = 'ChainageS'
    rawHtml0.loc[2,7] = 'ChainageE'

    # select only table information
    html_data0         = rawHtml0.loc[3:len(rawHtml0),:] 

    # change column name
    html_data0.columns = rawHtml0.loc[2,:]
    html_data0 = html_data0.reset_index(drop=True)
    html_data0['width'] = np.nan
    html_data0['nrLanes'] = np.nan
    html_data0 = html_data0.rename(index=str, columns={"(Km)": "Distance", "(AADT)": "AADT"})
    html_data0 = html_data0.drop('Total AADT', axis=1)
    html_data0.columns.values[0] = 'road'
    html_data0.columns.values[2] = 'LRPStart'
    html_data0.columns.values[3] = 'OffsetStart'
    html_data0.columns.values[5] = 'LRPEnd'
    html_data0.columns.values[6] = 'OffsetEnd'
       
    notfound = []
    
    # import txt width lanes files
    for i in range (1,len(bdf_orig.road.value_counts().index)):
        rname = bdf_orig.road.value_counts().index[i]
        try:
            textPath = path + '\\' + rname + '.widths.processed.txt'
            width_data      = pd.read_table(textPath)

            # reading raw html
            htmlPath = path + '\\' + rname + ".traffic.htm"
            rawHtml  = pd.read_html(htmlPath)[4]

            rawHtml.loc[2,0] = 'LinkNo'
            rawHtml.loc[2,1] = 'LinkName'
            rawHtml.loc[2,4] = 'ChainageS'
            rawHtml.loc[2,7] = 'ChainageE'

            # select only table information
            html_data         = rawHtml.loc[3:len(rawHtml),:] 

            # change column name
            html_data.columns = rawHtml.loc[2,:]
            html_data = html_data.reset_index(drop=True)
            html_data['width'] = np.nan
            html_data['nrLanes'] = np.nan
            html_data = html_data.rename(index=str, columns={"(Km)": "Distance", "(AADT)": "AADT"})
            html_data = html_data.drop('Total AADT', axis=1)
            html_data.columns.values[0] = 'road'
            html_data.columns.values[2] = 'LRPStart'
            html_data.columns.values[3] = 'OffsetStart'
            html_data.columns.values[5] = 'LRPEnd'
            html_data.columns.values[6] = 'OffsetEnd'

            width_data0  = pd.concat([width_data0, width_data])
            html_data0  = pd.concat([html_data0, html_data])
            
        except:
            notfound.append(bdf_orig.road.value_counts().index[i])
    
    return df_orig,bdf_orig,flood_shhp_data,flood_geoPath,width_data0,html_data0,notfound


In [3]:
def selected_data(rname,df,bdf,width_data_final,html_data_final):
    # Create dataframe for selected roads for BMMS and roads data
    df_selected  = df[(df['road'] == rname)].sort_values(by = 'chainage')
    bdf_selected = bdf[(bdf['road'] == rname)].sort_values(by = 'chainage')

    df_selected['width'] = np.nan
    df_selected['Length'] = np.nan
    df_selected['Quality'] = np.nan
    df_selected['LinkName'] = np.nan
    df_selected['constructionYear'] = np.nan

    # mark which data is from which source
    df_selected['Data_source'] = 'road3'
    bdf_selected['Data_source'] = 'BMMS'

    # drop unnecessary column that is not needed in the construction of excel files
    df_selected = df_selected.drop(['gap'], axis=1)
    bdf_selected = bdf_selected.drop(['km', 'structureNr', 'spans', 
                                      'zone','circle','division',
                                      'sub-division', 'EstimatedLoc'], axis=1)

    # Script to drop "Bridges" included in the roads data
    df_selected = df_selected.drop(df_selected[df_selected['type'] == 'Bridge'].index)

    # rename some columns names
    bdf_selected = bdf_selected.rename(index=str, columns={'LRPName': 'lrp' ,'condition':'Quality',
                                                           'length':'Length', 'roadName':'LinkName'})

    # Combined roads and bridges data
    df1_combined = pd.concat([df_selected, bdf_selected]).sort_values(by = 'chainage')

    # drop duplicates
    df1_combined = df1_combined.drop_duplicates(subset = ['lrp'], keep = 'first', inplace=False)
    df1_combined = df1_combined.sort_values(by = 'chainage').reset_index(drop = True)
    
    # Filling the quality of the road based on the quality of the bridge (FIRST TRY, PLEASE REMOVE IT AFTERWARDS)
    df1_combined['Quality'] = df1_combined.Quality.fillna(method='bfill')
    
    # selected dataframe for width data
    width_data_final_selected  = width_data_final[(width_data_final['roadNo'] == rname)].sort_values(by = 'startChainage')

    # selected dataframe for html data
    # deleting character name after "-" character to the data
    html_data_final['road'] = html_data_final['road'].apply(lambda x: x[:x.rfind("-")])
    html_data_final_selected  = html_data_final[(html_data_final['road'] == rname)].sort_values(by = 'ChainageS')
    
    return df1_combined,width_data_final_selected,html_data_final_selected

In [4]:
def plot_map(df1_combined):
# Load map centred on average coordinates

    my_map = folium.Map(location=[df1_combined.lat.median(), df1_combined.lon.median()], zoom_start=10)

    for i in range(len(df1_combined)-1):
        points = []
        if (df1_combined['Quality'][i]=='A'):
            points.append(tuple([df1_combined.lat[i], df1_combined.lon[i]]))
            points.append(tuple([df1_combined.lat[i+1], df1_combined.lon[i+1]]))
            folium.PolyLine(points, color="green", weight=2.5, opacity=1).add_to(my_map)
        if (df1_combined['Quality'][i]=='B'):
            points.append(tuple([df1_combined.lat[i], df1_combined.lon[i]]))
            points.append(tuple([df1_combined.lat[i+1], df1_combined.lon[i+1]]))
            folium.PolyLine(points, color="yellow", weight=2.5, opacity=1).add_to(my_map)
        if (df1_combined['Quality'][i]=='C'):
            points.append(tuple([df1_combined.lat[i], df1_combined.lon[i]]))
            points.append(tuple([df1_combined.lat[i+1], df1_combined.lon[i+1]]))
            folium.PolyLine(points, color="orange", weight=2.5, opacity=1).add_to(my_map)
        if (df1_combined['Quality'][i]=='D'):
            points.append(tuple([df1_combined.lat[i], df1_combined.lon[i]]))
            points.append(tuple([df1_combined.lat[i+1], df1_combined.lon[i+1]]))
            folium.PolyLine(points, color="red", weight=2.5, opacity=1).add_to(my_map)


    for i in range(len(df1_combined)):
        if (df1_combined['Data_source'][i] == 'BMMS'):
            points_bridges = []
            if(df1_combined['Quality'][i]=='A'):
                points_bridges.append(tuple([df1_combined.lat[i], df1_combined.lon[i]]))
                testa = folium.Html('<b>A bridges!</b>', script=True)
                popupa = folium.Popup(testa, max_width=2650)
                folium.RegularPolygonMarker(points_bridges[0],popup=popupa,fill_color='#61ff00',
                                            number_of_sides=4,radius=8).add_to(my_map)

            if(df1_combined['Quality'][i]=='B'):
                points_bridges.append(tuple([df1_combined.lat[i], df1_combined.lon[i]]))
                testb = folium.Html('<b>B bridges!</b>', script=True)
                popupb = folium.Popup(testb, max_width=2650)
                folium.RegularPolygonMarker(points_bridges[0],popup=popupb,fill_color='#fffa00',
                                            number_of_sides=4,radius=8).add_to(my_map)

            if(df1_combined['Quality'][i]=='C'):
                points_bridges.append(tuple([df1_combined.lat[i], df1_combined.lon[i]]))
                testc = folium.Html('<b>C bridges!</b>', script=True)
                popupc = folium.Popup(testc, max_width=2650)
                folium.RegularPolygonMarker(points_bridges[0],popup=popupc,fill_color='#ffa100',
                                            number_of_sides=4,radius=8).add_to(my_map)

            if(df1_combined['Quality'][i]=='D'):
                points_bridges.append(tuple([df1_combined.lat[i], df1_combined.lon[i]]))
                testd = folium.Html('<b>D bridges!</b>', script=True)
                popupd = folium.Popup(testd, max_width=2650)
                folium.RegularPolygonMarker(points_bridges[0],popup=popupd,fill_color='#ff0000',
                                            number_of_sides=4,radius=8).add_to(my_map)

    return my_map

In [5]:
def function1(rname,road_path,bridge_path,fp,path):
    df_orig,bdf_orig,flood_shhp_data,flood_geoPath,width_data0,html_data0,notfound = import_data(road_path,bridge_path,fp,path)
    df1_combined,width_data_final_selected,html_data_final_selected = selected_data(rname,df_orig,bdf_orig,width_data0,html_data0)
    mymap  = plot_map(df1_combined)
    return mymap

In [6]:
rname = 'N1' # name of the road to load
mymap1  = function1(rname,road_path,bridge_path,fp,path)
mymap1

In [7]:
display(mymap1,mymap2,mymap3)

NameError: name 'mymap2' is not defined

In [ ]:
for i in range(0,len(trafficDF)-1):
        if trafficDF.iloc[i,4]  == trafficDF.iloc[i+1,4]:
            for j in range(9,len(trafficDF.columns)):
                trafficDF.iloc[i+1,j] = float(trafficDF.iloc[i,j]) + float(trafficDF.iloc[i+1,j])
                trafficDF.iloc[i+1,j] = str(trafficDF.iloc[i+1,j])
    trafficDF = trafficDF.drop_duplicates(subset='chainage',keep='last')
    trafficDF = trafficDF.reset_index(drop=True)

In [ ]:
# widgets.SelectMultiple(options=list(bdf_orig.road.value_counts().index.sort_values()),
#                                    value=['N1'],
#                                    rows=10,description='Road',disabled=False)
# widgets.interact(function1,rname = 'N1',df_orig = df_orig,bdf_orig = bdf_orig,width_data0=width_data0 ,html_data0=html_data0)